In [1]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1, 1))
display.start()

import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import gym
import numpy as np
import cv2

In [2]:
from stable_baselines.common.policies import MlpLstmPolicy, MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines import TD3
from stable_baselines.common import make_vec_env, set_global_seeds

/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: P

In [3]:
def evaluate_2(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes
    :return: (float) Mean reward for the given number of episodes
    """
    episode_rewards = []
    obs = env.reset()
    for i in range(num_episodes):
        episode_rewards.append(0.0)
        done = False
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            reward, done, info = reward[0], done[0], info[0]
            # Stats
            episode_rewards[-1] += reward
            if done:
                obs = env.reset()
    # Compute mean reward for the last 100 episodes
    mean_100ep_reward = round(np.mean(episode_rewards), 1)
    print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards))
    return mean_100ep_reward

In [4]:
def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        env.seed(seed + rank)
        return env
    set_global_seeds(seed)
    return _init


# Stable Baselines provides you with make_vec_env() helper
# which does exactly the previous steps for you:
# env = make_vec_env(env_id, n_envs=num_cpu, seed=0)

In [5]:
from stable_baselines.common.policies import FeedForwardPolicy, register_policy

# Custom MLP policy of three layers of size 128 each
class CustomPolicy(FeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomPolicy, self).__init__(*args, **kwargs,
                                           net_arch=[dict(pi=[64, 64],
                                                          vf=[64, 64, 32])],
                                           feature_extraction="mlp")

# Register the policy, it will check that the name is not already taken
register_policy('CustomPolicy', CustomPolicy)

# Because the policy is now registered, you can pass
# a string to the agent constructor instead of passing a class
# model = A2C(policy='CustomPolicy', env='LunarLander-v2', verbose=1).learn(total_timesteps=100000)


In [11]:
env_id = "BipedalWalker-v2"
env = DummyVecEnv([lambda: gym.make(env_id)])

In [ ]:
%%time
from stable_baselines.td3.policies import MlpPolicy as MlpPolicyTd
model = TD3(MlpPolicyTd, env, verbose=1, buffer_size=200000, batch_size=5000, learning_starts=50, random_exploration = 0.1)
model.learn(total_timesteps=200000, log_interval=10)

---------------------------------------
| current_lr              | 0.0003    |
| episodes                | 10        |
| fps                     | 92        |
| mean 100 episode reward | -88.7     |
| n_updates               | 2300      |
| qf1_loss                | 2.887241  |
| qf2_loss                | 3.1945798 |
| time_elapsed            | 78        |
| total timesteps         | 7277      |
---------------------------------------
--------------------------------------
| current_lr              | 0.0003   |
| episodes                | 20       |
| fps                     | 69       |
| mean 100 episode reward | -104     |
| n_updates               | 3500     |
| qf1_loss                | 5.233383 |
| qf2_loss                | 6.190755 |
| time_elapsed            | 121      |
| total timesteps         | 8486     |
--------------------------------------
---------------------------------------
| current_lr              | 0.0003    |
| episodes                | 30        |
| fps      

In [9]:
model.save("td3_custom")

In [10]:
env = gym.make('BipedalWalker-v2')
env = DummyVecEnv([lambda: env])
model = ACKTR.load("td3_custom", env=env, nminibatches=16)

In [11]:
evaluate_2(model, 100)

Mean reward: 60.8 Num episodes: 100


60.8

## mp train for 1M ts

In [13]:
env_id = "BipedalWalker-v2"
num_cpu = 10  # Number of processes to use
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

In [14]:
model = ACKTR.load("acktr_custom_parallel", env=env, verbose=1)

In [15]:
%%time
model.learn(total_timesteps=500000, log_interval=10)

---------------------------------
| explained_variance | 0.44     |
| fps                | 284      |
| nupdates           | 1        |
| policy_entropy     | 5.25     |
| policy_loss        | -0.00462 |
| total_timesteps    | 0        |
| value_loss         | 3.75     |
---------------------------------
---------------------------------
| explained_variance | 0.087    |
| fps                | 1624     |
| nupdates           | 10       |
| policy_entropy     | 5.26     |
| policy_loss        | -0.0719  |
| total_timesteps    | 1809     |
| value_loss         | 1.6      |
---------------------------------
---------------------------------
| explained_variance | 0.107    |
| fps                | 2298     |
| nupdates           | 20       |
| policy_entropy     | 5.26     |
| policy_loss        | -0.0849  |
| total_timesteps    | 3819     |
| value_loss         | 4.56     |
---------------------------------
---------------------------------
| explained_variance | 0.715    |
| fps         

In [16]:
model.save("acktr_custom_parallel_1M")

In [17]:
env = gym.make('BipedalWalker-v2')
env = DummyVecEnv([lambda: env])
model = ACKTR.load("acktr_custom_parallel_1M", env=env, nminibatches=16)

In [18]:
evaluate_2(model, 100)

Mean reward: 152.7 Num episodes: 100


152.7